In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
train_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor())



Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())
type(test_data)

torchvision.datasets.mnist.FashionMNIST

In [4]:
batch_size = 64

# 데이터로더를 생성합니다.
train_dataloader = DataLoader(train_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
train = pd.DataFrame(train_dataloader)

for X, y in train_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

type(train_dataloader)

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


torch.utils.data.dataloader.DataLoader

In [5]:
# 학습에 사용할 CPU나 GPU 장치를 얻습니다.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    # 층 만듬
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.309202  [    0/60000]
loss: 2.301596  [ 6400/60000]
loss: 2.294254  [12800/60000]
loss: 2.288796  [19200/60000]
loss: 2.275445  [25600/60000]
loss: 2.271825  [32000/60000]
loss: 2.268940  [38400/60000]
loss: 2.255210  [44800/60000]
loss: 2.273706  [51200/60000]
loss: 2.247725  [57600/60000]
Test Error: 
 Accuracy: 31.2%, Avg loss: 0.035276 

Epoch 2
-------------------------------
loss: 2.255664  [    0/60000]
loss: 2.257086  [ 6400/60000]
loss: 2.219341  [12800/60000]
loss: 2.227231  [19200/60000]
loss: 2.198795  [25600/60000]
loss: 2.190339  [32000/60000]
loss: 2.190347  [38400/60000]
loss: 2.153822  [44800/60000]
loss: 2.202249  [51200/60000]
loss: 2.167446  [57600/60000]
Test Error: 
 Accuracy: 33.2%, Avg loss: 0.033708 

Epoch 3
-------------------------------
loss: 2.163704  [    0/60000]
loss: 2.160317  [ 6400/60000]
loss: 2.074957  [12800/60000]
loss: 2.108896  [19200/60000]
loss: 2.066613  [25600/60000]
loss: 2.056710  [32000/600

In [ ]:
test(test_dataloader, model)

Test Error: 
 Accuracy: 58.8%, Avg loss: 0.022753 



In [ ]:
tensor = torch.rand(3,6)
print(tensor)
print()
lis = torch.chunk(tensor, 6, dim=0)
print(lis)

tensor([[0.9688, 0.4024, 0.6783, 0.5342, 0.9759, 0.6677],
        [0.7210, 0.7229, 0.1332, 0.8741, 0.9853, 0.0122],
        [0.0260, 0.1079, 0.7778, 0.4940, 0.1785, 0.6635]])

(tensor([[0.9688, 0.4024, 0.6783, 0.5342, 0.9759, 0.6677]]), tensor([[0.7210, 0.7229, 0.1332, 0.8741, 0.9853, 0.0122]]), tensor([[0.0260, 0.1079, 0.7778, 0.4940, 0.1785, 0.6635]]))
